The point of this notebook is to test whether the explained variance we witness
can be attributed to randomness

Process :
- For each layer : 
         -Do a hundred time :
          Generate a feature vector from an untrained Soundnet
          - For each fold :
              Use that feature vector to train a MLP on a given subject
              Use the MLP to predict brain activity
              Plot the r2_scores comparing prediction to truth
              Save the best score accross the whole brain   

In [1]:
### Remove useless warnings from scikit learn
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [2]:
### Add the stage folder to path in order to import feature vectors extraction
import sys
origin_dir = "/home/brain/victor/stage/fmri_video/code"
new_dir =  '/home/brain/victor/stage/'
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/brain/victor/stage/')

In [3]:
save_feature_vectors = "/home/brain/victor/stage/fmri_video/soundnet_features/baseline/conv{}.npz"
stimulus_location ='/home/brain/victor/stage/fmri_video/stimulus/sherlockaudio_22k.wav'


In [42]:
import feature_extraction_weightless as feature_extractor
import numpy as np
from matplotlib import pyplot as plt
from nilearn.plotting import plot_roi, plot_stat_map, plot_anat
from nilearn.image import mean_img
import os.path
from joblib import dump, load
import pandas as pd

In [5]:
from sklearn.model_selection import cross_validate, GridSearchCV, KFold
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
import os
import tqdm

In [6]:
def extract_one_vector():
    os.chdir(new_dir)
    """Extract 1 random feature vector for layers {5,6,7}"""
    vector, size =feature_extractor.extract_vectors(stimulus_location)
    for id_layer in range(5,8):
        feature = feature_extractor.quantize_test(vector[id_layer-1], id_layer-1,1.5, size)
        np.savez_compressed(save_feature_vectors.format(id_layer),fv = feature)
        print("Extracted and saved weightless feature vector for conv{}".format(id_layer))

In [7]:
def load_feature_vector(layer_id = 7) :
    file_fv = save_feature_vectors.format(layer_id)
    fv = np.load(file_fv)['fv']
    # Check the size
    print("layer {}, {} frames, FV dimension is {}".format(layer_id,fv.shape[0], fv.shape[1]))
    return fv

In [8]:
def load_fmri(id_subject = 1):
    fmri_ready_path = os.path.join(origin_dir, "fmri_ready/subject{:02d}.npy".format(id_subject))
    fmri_ready = np.load(fmri_ready_path)
    fmri_mean_path = os.path.join(origin_dir, "fmri_mean/subject{:02d}.npy".format(id_subject))
    fmri_mean = fmri_mean_path
    ward_path = os.path.join(origin_dir, "fmri_parcellated/subject{:02d}.nii.gz".format(id_subject))
    ward = load(ward_path)
    return fmri_ready, fmri_mean, ward

In [9]:
def load_data(id_layer,id_subject): 
    X = load_feature_vector(id_layer)
    fv_shape = X.shape[0]  # need the feature vector shape to resize fmri data
    y, fmri_mean, ward = load_fmri(id_subject)
    return X, y, fmri_mean, ward

In [10]:
def train(X, y, fmri_mean, ward, n_folds,id_layer, id_subject, id_experience = 1):
    print("Training on subject {} and layer {} with {} folds".format(id_subject, id_layer, n_folds))
    mlp_estimator = MLPRegressor(hidden_layer_sizes=(1000,),solver='adam',activation='relu',
                             max_iter=2000,learning_rate_init=0.001, alpha=0.0001, 
                             batch_size=50,early_stopping=True,verbose=False, warm_start = False)
    cv = KFold(n_splits = n_folds)
    fold_number = 1
    scores = []
    for train_index, test_index in cv.split(X):
        print("Subject {} Fold {},layer {} index = [{},{}]:".format(id_subject,fold_number,id_layer, test_index[0], test_index[-1]))
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # warm_start is False so each train is fitted on a new MLP
        mlp_estimator.fit(X_train, y_train) 
        predictions = mlp_estimator.predict(X_test)
        r2_scores = r2_score(y_test, predictions, multioutput="raw_values")
        r2_scores[r2_scores < 0 ] = 0
        scores_img = ward.inverse_transform(r2_scores.reshape((1,-1)))
        r2_max = np.max(r2_scores)
        print(r2_max)
        scores_img.to_filename("{}fold{}_sub{:02d}_exp{}.nii.gz".format(layer_result_folder, fold_number, id_subject,id_experience))
        fold_number += 1
        scores.append(r2_max)
    return scores

In [14]:
n_experience = 100
n_neurons = 1000
id_subject = 1
filename = "weightless_scores.pkl"

for id_experience in tqdm.tqdm(range(n_experience)):
    print("Experience #{}".format(id_experience))
    if filename in os.listdir(".") :
            weightless_scores = load(filename)
            print("Loaded weightless scores, with length : {}".format(len(weightless_scores)))
    else :
        weightless_scores = {}
    extract_one_vector()
    experience_scores = {}
    for id_layer in range(5,8):
        n_folds = 4
        result_folder = "/home/brain/victor/stage/fmri_video/code/results/victor/parcellation_conv{}_weightless/"
        print("Layer #{}".format(id_layer))
        layer_result_folder = result_folder.format(id_layer)
        layer_scores = {}
        if id_subject != 5 :
            X, y, fmri_mean, ward = load_data(id_layer,id_subject)
            r2_scores = train(X, y, fmri_mean, ward, n_folds, id_layer, id_subject, id_experience)
            experience_scores[id_layer] = list(r2_scores)
    weightless_scores[id_experience] = dict(experience_scores)
    dump(weightless_scores,filename)

  0%|          | 0/100 [00:00<?, ?it/s]

Experience #0
Loaded weightless scores, with length : 3
time:  0.07779264450073242
Cout :  16
Cout :  32
Cout :  64
Cout :  128
Cout :  256
Cout :  512
Cout :  1024
Cout :  1000
Cout :  401
Extracted and saved weightless feature vector for conv5
Extracted and saved weightless feature vector for conv6
Extracted and saved weightless feature vector for conv7
Layer #5
layer 5, 946 frames, FV dimension is 256
Training on subject 1 and layer 5 with 4 folds
Subject 1 Fold 1,layer 5 index = [0,236]:
8.657718218119115e-06
Subject 1 Fold 2,layer 5 index = [237,473]:
1.4973066329027063e-06
Subject 1 Fold 3,layer 5 index = [474,709]:
0.0
Subject 1 Fold 4,layer 5 index = [710,945]:
0.0
Layer #6
layer 6, 946 frames, FV dimension is 512
Training on subject 1 and layer 6 with 4 folds
Subject 1 Fold 1,layer 6 index = [0,236]:
0.0
Subject 1 Fold 2,layer 6 index = [237,473]:
5.357153249896385e-07
Subject 1 Fold 3,layer 6 index = [474,709]:
0.0
Subject 1 Fold 4,layer 6 index = [710,945]:
0.0
Layer #7
laye

  1%|          | 1/100 [01:43<2:50:17, 103.21s/it]

0.0
Experience #1
Loaded weightless scores, with length : 4
time:  0.06764650344848633
Cout :  16
Cout :  32
Cout :  64
Cout :  128
Cout :  256
Cout :  512
Cout :  1024
Cout :  1000
Cout :  401
Extracted and saved weightless feature vector for conv5
Extracted and saved weightless feature vector for conv6
Extracted and saved weightless feature vector for conv7
Layer #5
layer 5, 946 frames, FV dimension is 256
Training on subject 1 and layer 5 with 4 folds
Subject 1 Fold 1,layer 5 index = [0,236]:
0.0
Subject 1 Fold 2,layer 5 index = [237,473]:
1.0183296739851855e-05
Subject 1 Fold 3,layer 5 index = [474,709]:
3.8546081705392865e-06
Subject 1 Fold 4,layer 5 index = [710,945]:
3.5862711488254817e-06
Layer #6
layer 6, 946 frames, FV dimension is 512
Training on subject 1 and layer 6 with 4 folds
Subject 1 Fold 1,layer 6 index = [0,236]:
0.0
Subject 1 Fold 2,layer 6 index = [237,473]:
5.804981052515146e-07
Subject 1 Fold 3,layer 6 index = [474,709]:
3.55674242136228e-06
Subject 1 Fold 4,lay

  2%|▏         | 2/100 [03:36<2:53:24, 106.17s/it]

...

0.0
Experience #98
Loaded weightless scores, with length : 100
time:  0.07454490661621094
Cout :  16
Cout :  32
Cout :  64
Cout :  128
Cout :  256
Cout :  512
Cout :  1024
Cout :  1000
Cout :  401
Extracted and saved weightless feature vector for conv5
Extracted and saved weightless feature vector for conv6
Extracted and saved weightless feature vector for conv7
Layer #5
layer 5, 946 frames, FV dimension is 256
Training on subject 1 and layer 5 with 4 folds
Subject 1 Fold 1,layer 5 index = [0,236]:
1.69960565521432e-05
Subject 1 Fold 2,layer 5 index = [237,473]:
0.0
Subject 1 Fold 3,layer 5 index = [474,709]:
5.6195103751432995e-06
Subject 1 Fold 4,layer 5 index = [710,945]:
9.005912465998023e-06
Layer #6
layer 6, 946 frames, FV dimension is 512
Training on subject 1 and layer 6 with 4 folds
Subject 1 Fold 1,layer 6 index = [0,236]:
2.617466574594829e-07
Subject 1 Fold 2,layer 6 index = [237,473]:
0.0
Subject 1 Fold 3,layer 6 index = [474,709]:
0.0
Subject 1 Fold 4,layer 6 index = [710

 99%|█████████▉| 99/100 [3:01:47<02:01, 121.75s/it]

0.0
Experience #99
Loaded weightless scores, with length : 101
time:  0.06749892234802246
Cout :  16
Cout :  32
Cout :  64
Cout :  128
Cout :  256
Cout :  512
Cout :  1024
Cout :  1000
Cout :  401
Extracted and saved weightless feature vector for conv5
Extracted and saved weightless feature vector for conv6
Extracted and saved weightless feature vector for conv7
Layer #5
layer 5, 946 frames, FV dimension is 256
Training on subject 1 and layer 5 with 4 folds
Subject 1 Fold 1,layer 5 index = [0,236]:
3.747404727660886e-05
Subject 1 Fold 2,layer 5 index = [237,473]:
6.920091636386516e-07
Subject 1 Fold 3,layer 5 index = [474,709]:
2.1979942277372544e-07
Subject 1 Fold 4,layer 5 index = [710,945]:
0.0
Layer #6
layer 6, 946 frames, FV dimension is 512
Training on subject 1 and layer 6 with 4 folds
Subject 1 Fold 1,layer 6 index = [0,236]:
0.0
Subject 1 Fold 2,layer 6 index = [237,473]:
0.0
Subject 1 Fold 3,layer 6 index = [474,709]:
4.4285262212895304e-07
Subject 1 Fold 4,layer 6 index = [7

100%|██████████| 100/100 [3:04:23<00:00, 131.80s/it]

0.0


In [15]:
weightless_scores = load(filename)

In [21]:
print(weightless_scores[0])

{5: [8.657718218119115e-06, 1.4973066329027063e-06, 0.0, 0.0], 6: [0.0, 5.357153249896385e-07, 0.0, 0.0], 7: [3.000572922351452e-06, 0.0, 0.0, 0.0]}


In [37]:
layers = dict({
5 : np.array([]),
6 : np.array([]),
7 : np.array([])})
for id_experience in weightless_scores.keys():
    for id_layer in range(5,8) :    
        layers[id_layer]  = np.append(layers[id_layer], weightless_scores[id_experience][id_layer])
    

In [45]:
print(np.std(layers[5]))
print(np.std(layers[6]))
print(np.std(layers[7]))

1.106963926582257e-05
2.3066909502076187e-06
2.6421579552634424e-06


In [ ]:
t = pd.DataFrame()
t.append

In [ ]:
\begin{tabular}{lrrrrr}
\toprule
{} &  conv5   &  conv6  &  conv7 \\
\midrule
mean &    7.1e-06 &    9.7e-07 &    8.4e-07 &\\
std &    1.1e-05 &   2.3e-06 &    2.6e-06 & \\
\bottomrule
\end{tabular}

